Implement a software algorithm using Python to identify transistor sizes in CMOS
combinational logic circuits. The transistor sizes should be selected in a way that meets minimum
area, makes tpHL = tpLH, and achieve minimum delay

Using Numpy libaray

General Code

In [2]:
import numpy as np
from scipy.optimize import minimize

def objective(x):        # Objective: minimize the overall delay while considering area and tpHL = tpLH

    return x[0] * (x[1] + x[2]) + x[3]

def constraint1(x):     # Constraint 1: Area Constraint

    return x[0] * (x[1] + x[2])

def constraint2(x):     # Constraint 2: tpHL = tpLH

    return x[1] * x[2] - x[3]

# initial guesses
n = 4
x0 = np.zeros(n)
x0[0] = 1.0  # Width of PMOS
x0[1] = 1.0  # Width of NMOS1
x0[2] = 1.0  # Width of NMOS2
x0[3] = 1.0  # Length of NMOS1 and NMOS2

# bounds for transistor sizes
bounds = [(1.0, 5.0), (0.1, 5.0), (0.1, 5.0), (0.1, 5.0)]

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# define constraints
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
constraints = [con1, con2]

# optimize
solution = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x)))

# print solution
print('Solution:')
print('Width of PMOS: ' + str(x[0]))
print('Width of NMOS1: ' + str(x[1]))
print('Width of NMOS2: ' + str(x[2]))
print('Length of NMOS1 and NMOS2: ' + str(x[3]))


Initial Objective: 3.0
Final Objective: 0.7324555330351467
Solution:
Width of PMOS: 1.0
Width of NMOS1: 0.3162277665175696
Width of NMOS2: 0.316227766517577
Length of NMOS1 and NMOS2: 0.1


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


This Code Is othe solution using Deap libarary

In [ ]:
pip install deap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
import random
from deap import base, creator, tools, algorithms

# Define problem as a minimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define transistor sizes (widths) for the circuit
MIN_WIDTH = 0.1
MAX_WIDTH = 10.0

# Define circuit parameters
Vdd = 1.8  # Supply voltage
Istatic_max = 1e-6  # Maximum static current
Cload =1
frequency=50

# Define the truth table for a 2-input NAND gate
truth_table = [
    (0, 0, 1),
    (0, 1, 1),
    (1, 0, 1),
    (1, 1, 0)
]

# Function to simulate the circuit and calculate fitness
def evaluate(individual):
    # Extract transistor sizes from the individual
    w1, w2, w3 = individual

    # Calculate dynamic power (Pdynamic) based on the truth table
    Pdynamic = 0.0
    for inputs in truth_table:
        A, B, output = inputs
        inputs_voltage = [A * Vdd, B * Vdd]
        output_voltage = output * Vdd

        # Simulate the circuit (you would replace this with your simulation method)
        # For simplicity, we assume an ideal model without considering delay
        output_simulated = (inputs_voltage[0] and inputs_voltage[1]) * Vdd

        # Calculate dynamic power based on the simulated output
        Pdynamic += 0.5 * Cload * Vdd**2 * frequency  # Modify as needed

    # Calculate static power (Pstatic)
    Pstatic = Istatic_max * Vdd

    # The fitness is the sum of dynamic and static power
    fitness = Pdynamic + Pstatic

    return (fitness,)

# Genetic Algorithm parameters
POP_SIZE = 10
NGEN = 10
CXPB = 0.7
MUTPB = 0.2

# Create the toolbox
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, MIN_WIDTH, MAX_WIDTH)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create an initial population
population = toolbox.population(n=POP_SIZE)

# Run the algorithm
algorithms.eaMuPlusLambda(population, toolbox, mu=POP_SIZE, lambda_=2*POP_SIZE, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=None, halloffame=None, verbose=True)

# Print the best individual at the end
best_ind = tools.selBest(population, k=1)[0]
print("Best Individual:", best_ind, "Fitness:", best_ind.fitness.values)


gen	nevals
0  	10    
1  	18    
2  	17    
3  	16    
4  	19    
5  	20    
6  	20    
7  	19    
8  	19    
9  	19    
10 	18    
Best Individual: [10.792036486570206, 6.511954050846534, 2.8729459200167793] Fitness: (324.0000018,)


In [ ]:
import random
from deap import base, creator, tools, algorithms

# Define problem as a minimization problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Define transistor sizes (widths)
MIN_WIDTH = 0.1
MAX_WIDTH = 10.0

# Define circuit parameters
Vdd = 1.8  # Supply voltage
Istatic_max = 1e-6  # Maximum static current
Cload =1
frequency = 50

# Define the truth table for a 4-input NAND gate
truth_table = [
    (0, 0, 0, 0, 1),
    (0, 0, 0, 1, 1),
    (0, 0, 1, 0, 1),
    (0, 0, 1, 1, 1),
    # ... (complete the truth table)
]

# Function to simulate the circuit and calculate fitness
def evaluate(individual):
    # Extract transistor sizes from the individual
    w1, w2, w3, w4, w5 = individual

    # Calculate dynamic power (Pdynamic) based on the truth table
    Pdynamic = 0.0
    for inputs in truth_table:
        inputs_voltage = [input_val * Vdd for input_val in inputs[:-1]]
        output_voltage = inputs[-1] * Vdd

        # Simulate the circuit (replace this with your simulation method)
        # For simplicity, we assume an ideal model without considering delay
        output_simulated = not all(inputs)

        # Calculate dynamic power based on the simulated output
        Pdynamic += 0.5 * Cload * Vdd**2 * frequency  # Modify as needed

    # Calculate static power (Pstatic)
    Pstatic = Istatic_max * Vdd

    # The fitness is the sum of dynamic and static power
    fitness = Pdynamic + Pstatic

    return (fitness,)

# Genetic Algorithm parameters
POP_SIZE = 10
NGEN = 10
CXPB = 0.7
MUTPB = 0.2

# Create the toolbox
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, MIN_WIDTH, MAX_WIDTH)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create an initial population
population = toolbox.population(n=POP_SIZE)

# Run the algorithm
algorithms.eaMuPlusLambda(population, toolbox, mu=POP_SIZE, lambda_=2*POP_SIZE, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=None, halloffame=None, verbose=True)

# Print the best individual at the end
best_ind = tools.selBest(population, k=1)[0]
print("Best Individual:", best_ind, "Fitness:", best_ind.fitness.values)


gen	nevals
0  	10    
1  	16    
2  	19    
3  	19    
4  	15    
5  	18    
6  	20    
7  	15    
8  	18    
9  	17    
10 	17    
Best Individual: [0.3795770626535392, -1.324493128670094, 3.300453225911898, 5.203688550882943, 8.337924255814645] Fitness: (324.0000018,)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [1]:
import numpy as np
from scipy.optimize import minimize

def objective(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[2]

def constraint1(x):
    return x[0]*x[1]*x[2]*x[3]-25.0

def constraint2(x):
    sum_eq = 40.0
    for i in range(4):
        sum_eq = sum_eq - x[i]**2
    return sum_eq

# initial guesses
n = 4
x0 = np.zeros(n)
x0[0] = 1.0
x0[1] = 5.0
x0[2] = 5.0
x0[3] = 1.0

# show initial objective
print('Initial SSE Objective: ' + str(objective(x0)))

# optimize
b = (1.0,5.0)
bnds = (b, b, b, b)
con1 = {'type': 'ineq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
cons = ([con1,con2])
solution = minimize(objective,x0,method='SLSQP',\
                    bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final SSE Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))
print('x3 = ' + str(x[2]))
print('x4 = ' + str(x[3]))

Initial SSE Objective: 16.0
Final SSE Objective: 17.01401724563517
Solution
x1 = 1.0
x2 = 4.742996096883977
x3 = 3.8211546234095715
x4 = 1.379407645075325
